This notebook simply goes to a number of positions in workd space and reads out the joint configugarion of the robot at the position.
These data is used in `scene_utils` to workaround the limitation of a missing exposed inverse kinematics solver as well as the missing `beam_to_cart_pos` method.
The data is collected in a grid pattern in the workspace of the robot. Currently only (0,1,0,0) is a supported orientation for the beam workaround.

In [1]:
from pathlib import Path
import json

import glfw
import tqdm

from alr_sim.sims.SimFactory import SimRepository
from alr_sim.core import Scene

from alr_sim_tools.scene_utils import reset_scene

pybullet build time: May  2 2023 05:55:04


In [2]:
FACTORY_SRTING = "mj_beta"
DT=0.0002
RENDER_MODE = Scene.RenderMode.HUMAN
CAMERA_POSITION = (0.5, 0.0, 1.0)
CAMERA_QUAT = (-0.70710678, 0, 0, 0.70710678)

MOVEMENT_DURATION = 2
WAIT_DURATIONS_BETWEEN_MOVEMENTS = 0.5
TARGET_QUAT = (0, 1, 0, 0)

# XS = [i*0.05 for i in range(4, 14)] # 0.2 to 0.6
# YS = [i*0.05 for i in range(-10, 12)] # -0.5 to 0.5
# ZS = [i*0.05 for i in range(2, 10)] # 0.1 to 0.5
XS = [i*0.05 for i in range(8,9)] # 0.1 to 0.6
YS = [i*0.05 for i in range(0, 1)] # -0.5 to 0.5
ZS = [i*0.05 for i in range(0, 1)] # 0.1 to 0.6


JOINT_VALUE_DECIMALS = 5
DATA_FILE = Path.cwd().parent / "alr_sim_tools" / "world2joint_coords.json"

In [3]:
positions = []

for x in XS:
    for y in YS:
        for z in ZS:
            positions.append((x, y, z))

positions


[(0.4, 0.0, 0.0)]

In [4]:
sim_factory = SimRepository.get_factory(FACTORY_SRTING)
scene = sim_factory.create_scene(object_list=[], dt=DT, render=RENDER_MODE)
agent = sim_factory.create_robot(scene, dt=DT)

scene.start()
scene.start_logging()

In [5]:
world_2_joint_coords = {}

for pos in tqdm.tqdm(positions):
    agent.gotoCartPositionAndQuat(pos, TARGET_QUAT, MOVEMENT_DURATION)
    agent.wait(WAIT_DURATIONS_BETWEEN_MOVEMENTS)
    world_2_joint_coords[pos] = tuple(agent.current_j_pos) 


100%|██████████| 1/1 [00:13<00:00, 13.54s/it]


In [6]:
scene.stop_logging()
reset_scene(FACTORY_SRTING, scene, agent)
glfw.terminate()

In [7]:

world_2_joint_coords_rounded = {
    (tuple(round(c, 2) for c in pos), tuple(TARGET_QUAT)): tuple(
        round(j, JOINT_VALUE_DECIMALS) for j in j_coords
    )
    for (pos, j_coords) in world_2_joint_coords.items()
}

world_2_joint_coords_rounded

{((0.4, 0.0, 0.0), (0, 1, 0, 0)): (-0.00021,
  0.41263,
  -4e-05,
  -2.65506,
  -1e-05,
  3.06489,
  0.78521)}

In [8]:
# convert to list of tuples of tuples

world_2_joint_coords_list = [(k, v) for k, v in world_2_joint_coords_rounded.items()]

world_2_joint_coords_list

[(((0.4, 0.0, 0.0), (0, 1, 0, 0)),
  (-0.00021, 0.41263, -4e-05, -2.65506, -1e-05, 3.06489, 0.78521))]

In [9]:
json.dump(world_2_joint_coords_list, open(DATA_FILE, "w"))